<a href="https://www.kaggle.com/code/vanshikaajindal/bert-lda?scriptVersionId=134990017" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# bert with lda
import os
import shutil
!pip install tensorflow
import tensorflow as tf
!pip install tensorflow_text

import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

data_folder = '/kaggle/input/text-summarization/data'

# Load train data
train_data = []
train_labels = []
class_names = []

train_folder = os.path.join(data_folder, 'train')
for class_folder in os.listdir(train_folder):
    if os.path.isdir(os.path.join(train_folder, class_folder)):
        class_names.append(class_folder)
        class_path = os.path.join(train_folder, class_folder)
        for file_name in os.listdir(class_path):
            if file_name.endswith('.txt'):
                file_path = os.path.join(class_path, file_name)
                with open(file_path, 'r') as file:
                    train_data.append(file.read())
                    train_labels.append(class_folder)

# Load test data
test_data = []
test_labels = []

test_folder = os.path.join(data_folder, 'test')
for class_folder in os.listdir(test_folder):
    if os.path.isdir(os.path.join(test_folder, class_folder)):
        class_path = os.path.join(test_folder, class_folder)
        for file_name in os.listdir(class_path):
            if file_name.endswith('.txt'):
                file_path = os.path.join(class_path, file_name)
                with open(file_path, 'r') as file:
                    test_data.append(file.read())
                    test_labels.append(class_folder)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score, precision_score
from sklearn.metrics import confusion_matrix

# Define BERT model
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'
bert_model_path = '/kaggle/input/bert-model-1'  # Replace with the actual file path
bert_preprocess_model_path = '/kaggle/input/preocessor'  # Replace with the actual file path

# Load BERT model and preprocessing model
bert_preprocess_model = hub.KerasLayer(bert_preprocess_model_path)
bert_model = hub.KerasLayer(bert_model_path)

# Preprocess the data using BERT
train_preprocessed = bert_preprocess_model(train_data)
train_embeddings = bert_model(train_preprocessed)['pooled_output']

# Convert BERT embeddings to document-term matrix using CountVectorizer
vectorizer = CountVectorizer()
train_doc_term_matrix = vectorizer.fit_transform(train_data)


In [3]:
# Apply LDA for dimensionality reduction
lda = LatentDirichletAllocation(n_components=10)  # Adjust the number of components as needed
train_lda_features = lda.fit_transform(train_doc_term_matrix)

# Convert labels to numeric format
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Train a classifier on LDA features
classifier = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(train_lda_features.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
classifier.fit(train_lda_features, train_labels_encoded, epochs=10)

# Preprocess and classify test data
test_preprocessed = bert_preprocess_model(test_data)
test_embeddings = bert_model(test_preprocessed)['pooled_output']
test_doc_term_matrix = vectorizer.transform(test_data)
test_lda_features = lda.transform(test_doc_term_matrix)
predictions = classifier.predict(test_lda_features)



Epoch 1/10
233/233 [==============================] - 1s 2ms/step - loss: 1.2167 - accuracy: 0.5253
Epoch 2/10
233/233 [==============================] - 1s 2ms/step - loss: 1.1208 - accuracy: 0.5583
Epoch 3/10
233/233 [==============================] - 1s 2ms/step - loss: 1.1005 - accuracy: 0.5599
Epoch 4/10
233/233 [==============================] - 1s 2ms/step - loss: 1.0891 - accuracy: 0.5697
Epoch 5/10
233/233 [==============================] - 1s 2ms/step - loss: 1.0783 - accuracy: 0.5668
Epoch 6/10
233/233 [==============================] - 1s 2ms/step - loss: 1.0707 - accuracy: 0.5699
Epoch 7/10
233/233 [==============================] - 1s 2ms/step - loss: 1.0643 - accuracy: 0.5716
Epoch 8/10
233/233 [==============================] - 1s 3ms/step - loss: 1.0551 - accuracy: 0.5739
Epoch 9/10
233/233 [==============================] - 1s 2ms/step - loss: 1.0504 - accuracy: 0.5704
Epoch 10/10
63/63 [==============================] - 0s 1ms/step


In [4]:
# # Convert predictions back to labels
# predicted_labels_encoded = tf.argmax(predictions, axis=1)
# predicted_labels = label_encoder.inverse_transform(predicted_labels_encoded.numpy())
# # Convert labels to numeric format
# label_encoder = LabelEncoder()
# train_labels_encoded = label_encoder.fit_transform(train_labels)
# test_labels_encoded = label_encoder.transform(test_labels)

# # Create and fit the OneHotEncoder
# one_hot_encoder = OneHotEncoder()
# one_hot_encoder.fit(train_labels_encoded.reshape(-1, 1))

# # Convert labels to one-hot encoded format
# train_labels_one_hot = one_hot_encoder.transform(train_labels_encoded.reshape(-1, 1)).astype(float)
# test_labels_one_hot = one_hot_encoder.transform(test_labels_encoded.reshape(-1, 1)).astype(float)

# # Train a classifier on LDA features
# classifier = Pipeline([
#     ('classifier', tf.keras.Sequential([
#         tf.keras.layers.Dense(256, activation='relu'),
#         tf.keras.layers.Dense(len(class_names), activation='softmax')
#     ]))
# ])

# classifier['classifier'].compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# classifier.fit(train_lda_features, tf.keras.utils.to_categorical(train_labels_encoded))

# # Preprocess and classify test data
# test_preprocessed = bert_preprocess_model(test_data)
# test_embeddings = bert_model(test_preprocessed)['pooled_output']
# test_doc_term_matrix = vectorizer.transform(test_data)
# test_lda_features = lda.transform(test_doc_term_matrix)
# predictions = classifier.predict(test_lda_features)

# # Convert predictions back to labels
# predicted_labels_encoded = tf.argmax(predictions, axis=1)
# predicted_labels = label_encoder.inverse_transform(predicted_labels_encoded.numpy())

# # Convert labels to numeric format
# test_labels_encoded = label_encoder.transform(test_labels)

# # Evaluate the classifier
# accuracy = tf.metrics.Accuracy()
# accuracy.update_state(test_labels_encoded, predicted_labels_encoded)
# loss = classifier['classifier'].evaluate(test_lda_features, tf.keras.utils.to_categorical(test_labels_encoded), verbose=0)[0]
# f1 = f1_score(test_labels, predicted_labels, average='weighted')
# precision = precision_score(test_labels, predicted_labels, average='weighted')
# confusion = confusion_matrix(test_labels, predicted_labels)

# print('Accuracy:', accuracy.result().numpy())
# print('Loss:', loss)
# print('F1 Score:', f1)
# print('Precision:', precision)
# print(confusion)

# Convert predictions back to labels
predicted_labels_encoded = tf.argmax(predictions, axis=1)
predicted_labels = label_encoder.inverse_transform(predicted_labels_encoded.numpy())

# Evaluate the classifier
accuracy = tf.metrics.Accuracy()
accuracy.update_state(test_labels_encoded, predicted_labels_encoded)
loss = classifier.evaluate(test_lda_features, test_labels_encoded, verbose=0)[0]
f1 = f1_score(test_labels, predicted_labels, average='weighted')
precision = precision_score(test_labels, predicted_labels, average='weighted')
confusion = confusion_matrix(test_labels, predicted_labels)

print('Accuracy:', accuracy.result().numpy())
print('Loss:', loss)
print('F1 Score:', f1)
print('Precision:', precision)
print(confusion)


Accuracy: 0.6208791
Loss: 0.9768496155738831
F1 Score: 0.575271181716926
Precision: 0.637121616489259
[[ 35  41  29 125   0]
 [ 13 193  18 253   0]
 [  0  10  89 157   1]
 [  4  40  22 916   0]
 [  0   0   0  46  10]]


In [5]:

# True Positives (TP)
tp = confusion.diagonal()

# False Positives (FP)
fp = confusion.sum(axis=0) - tp

# True Negatives (TN)
tn = confusion.sum() - (tp + fp + confusion.diagonal())

# False Negatives (FN)
fn = confusion.sum(axis=1) - tp

print('True Positives:', tp)
print('False Positives:', fp)
print('True Negatives:', tn)
print('False Negatives:', fn)

True Positives: [ 35 193  89 916  10]
False Positives: [ 17  91  69 581   1]
True Negatives: [1915 1525 1755 -411 1981]
False Negatives: [195 284 168  66  46]


In [6]:
# bert with nmf
import os
import shutil
!pip install tensorflow
import tensorflow as tf
!pip install tensorflow_text

import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

# Example usage
data_folder = '/kaggle/input/text-summarization/data'

# Load train data
train_data = []
train_labels = []
class_names = []

train_folder = os.path.join(data_folder, 'train')
for class_folder in os.listdir(train_folder):
    if os.path.isdir(os.path.join(train_folder, class_folder)):
        class_names.append(class_folder)
        class_path = os.path.join(train_folder, class_folder)
        for file_name in os.listdir(class_path):
            if file_name.endswith('.txt'):
                file_path = os.path.join(class_path, file_name)
                with open(file_path, 'r') as file:
                    train_data.append(file.read())
                    train_labels.append(class_folder)

# Load test data
test_data = []
test_labels = []

test_folder = os.path.join(data_folder, 'test')
for class_folder in os.listdir(test_folder):
    if os.path.isdir(os.path.join(test_folder, class_folder)):
        class_path = os.path.join(test_folder, class_folder)
        for file_name in os.listdir(class_path):
            if file_name.endswith('.txt'):
                file_path = os.path.join(class_path, file_name)
                with open(file_path, 'r') as file:
                    test_data.append(file.read())
                    test_labels.append(class_folder)

import tensorflow as tf
import tensorflow_hub as hub
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score, precision_score
from sklearn.metrics import confusion_matrix

# Define BERT model
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'
bert_model_path = '/kaggle/input/bert-model-1'  # Replace with the actual file path
bert_preprocess_model_path = '/kaggle/input/preocessor'  # Replace with the actual file path

# Load BERT model and preprocessing model
bert_preprocess_model = hub.KerasLayer(bert_preprocess_model_path)
bert_model = hub.KerasLayer(bert_model_path)

# Preprocess the data using BERT
train_preprocessed = bert_preprocess_model(train_data)
train_embeddings = bert_model(train_preprocessed)['pooled_output']

# Convert BERT embeddings to document-term matrix using CountVectorizer
vectorizer = CountVectorizer()
train_doc_term_matrix = vectorizer.fit_transform(train_data)

# Apply NMF for dimensionality reduction
nmf = NMF(n_components=10, random_state=42)  # Adjust the number of components as needed
train_nmf_features = nmf.fit_transform(train_doc_term_matrix)

# Convert labels to numeric format
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Train a classifier on BERT + NMF features
classifier = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(train_nmf_features.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),  
    tf.keras.layers.Dropout(0.5),                  
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
classifier.fit(train_nmf_features, train_labels_encoded, epochs=20)

# Preprocess and classify test data
test_preprocessed = bert_preprocess_model(test_data)
test_embeddings = bert_model(test_preprocessed)['pooled_output']
test_doc_term_matrix = vectorizer.transform(test_data)
test_nmf_features = nmf.transform(test_doc_term_matrix)
predictions = classifier.predict(test_nmf_features)

# Convert predictions back to labels
predicted_labels_encoded = tf.argmax(predictions, axis=1)
predicted_labels = label_encoder.inverse_transform(predicted_labels_encoded.numpy())

# Evaluate the classifier
accuracy = tf.metrics.Accuracy()
accuracy.update_state(test_labels_encoded, predicted_labels_encoded)
loss = classifier.evaluate(test_nmf_features, test_labels_encoded, verbose=0)[0]
f1 = f1_score(test_labels, predicted_labels, average='weighted')
precision = precision_score(test_labels, predicted_labels, average='weighted')
confusion = confusion_matrix(test_labels, predicted_labels)

print('Accuracy:', accuracy.result().numpy())
print('Loss:', loss)
print('F1 Score:', f1)
print('Precision:', precision)
print(confusion)



/opt/conda/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


Epoch 1/20
233/233 [==============================] - 2s 3ms/step - loss: 1.0853 - accuracy: 0.5556
Epoch 2/20
233/233 [==============================] - 1s 2ms/step - loss: 0.8474 - accuracy: 0.6621
Epoch 3/20
233/233 [==============================] - 1s 2ms/step - loss: 0.8128 - accuracy: 0.6863
Epoch 4/20
233/233 [==============================] - 1s 2ms/step - loss: 0.7911 - accuracy: 0.6980
Epoch 5/20
233/233 [==============================] - 1s 2ms/step - loss: 0.7760 - accuracy: 0.7062
Epoch 6/20
233/233 [==============================] - 1s 2ms/step - loss: 0.7633 - accuracy: 0.7151
Epoch 7/20
233/233 [==============================] - 1s 2ms/step - loss: 0.7566 - accuracy: 0.7149
Epoch 8/20
233/233 [==============================] - 1s 2ms/step - loss: 0.7530 - accuracy: 0.7172
Epoch 9/20
233/233 [==============================] - 1s 2ms/step - loss: 0.7424 - accuracy: 0.7237
Epoch 10/20
233/233 [==============================] - 1s 2ms/step - loss: 0.7350 - accuracy: 0.7250

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [7]:
# bert with svm
import os
import shutil
!pip install tensorflow
import tensorflow as tf
!pip install tensorflow_text

import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC

# Example usage
data_folder = '/kaggle/input/text-summarization/data'

# Load train data
train_data = []
train_labels = []
class_names = []

train_folder = os.path.join(data_folder, 'train')
for class_folder in os.listdir(train_folder):
    if os.path.isdir(os.path.join(train_folder, class_folder)):
        class_names.append(class_folder)
        class_path = os.path.join(train_folder, class_folder)
        for file_name in os.listdir(class_path):
            if file_name.endswith('.txt'):
                file_path = os.path.join(class_path, file_name)
                with open(file_path, 'r') as file:
                    train_data.append(file.read())
                    train_labels.append(class_folder)

# Load test data
test_data = []
test_labels = []

test_folder = os.path.join(data_folder, 'test')
for class_folder in os.listdir(test_folder):
    if os.path.isdir(os.path.join(test_folder, class_folder)):
        class_path = os.path.join(test_folder, class_folder)
        for file_name in os.listdir(class_path):
            if file_name.endswith('.txt'):
                file_path = os.path.join(class_path, file_name)
                with open(file_path, 'r') as file:
                    test_data.append(file.read())
                    test_labels.append(class_folder)

import tensorflow as tf
import tensorflow_hub as hub
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score, precision_score
from sklearn.metrics import confusion_matrix

# Define BERT model
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'
tfhub_handle_encoder = '/kaggle/input/bert-model-1'
tfhub_handle_preprocess = '/kaggle/input/preocessor'

bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
bert_model = hub.KerasLayer(tfhub_handle_encoder)

# Preprocess the data using BERT
train_preprocessed = bert_preprocess_model(train_data)
train_embeddings = bert_model(train_preprocessed)['pooled_output']

# Convert BERT embeddings to document-term matrix using CountVectorizer
vectorizer = CountVectorizer()
train_doc_term_matrix = vectorizer.fit_transform(train_data)

# Apply NMF for dimensionality reduction
nmf = NMF(n_components=10, random_state=42)  # Increase the number of components to 20
train_nmf_features = nmf.fit_transform(train_doc_term_matrix)

# Convert labels to numeric format
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Train an SVM classifier on BERT + NMF features
classifier = SVC(kernel='linear')
classifier.fit(train_nmf_features, train_labels_encoded)

# Preprocess and classify test data
test_preprocessed = bert_preprocess_model(test_data)
test_embeddings = bert_model(test_preprocessed)['pooled_output']
test_doc_term_matrix = vectorizer.transform(test_data)
test_nmf_features = nmf.transform(test_doc_term_matrix)
predictions = classifier.predict(test_nmf_features)

# Convert predictions back to labels
predicted_labels = label_encoder.inverse_transform(predictions)

# Evaluate the classifier
accuracy = tf.metrics.Accuracy()
accuracy.update_state(test_labels_encoded, predictions)
f1 = f1_score(test_labels, predicted_labels, average='weighted')
precision = precision_score(test_labels, predicted_labels, average='weighted')
confusion = confusion_matrix(test_labels, predicted_labels)

print('Accuracy:', accuracy.result().numpy())
print('F1 Score:', f1)
print('Precision:', precision)
print(confusion)



/opt/conda/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


Accuracy: 0.6753247
F1 Score: 0.6384406055693846
Precision: 0.6684363609017894
[[  2 175  39  14   0]
 [  4 421  49   3   0]
 [  1 170  77   9   0]
 [  0 103  27 852   0]
 [  0  54   2   0   0]]


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
# bert with hdp
import os
import shutil
import tensorflow as tf
!pip install tensorflow_hub
import tensorflow_hub as hub
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

# Example usage
data_folder = '/kaggle/input/text-summarization/data'

# Load train data
train_data = []
train_labels = []
class_names = []

train_folder = os.path.join(data_folder, 'train')
for class_folder in os.listdir(train_folder):
    if os.path.isdir(os.path.join(train_folder, class_folder)):
        class_names.append(class_folder)
        class_path = os.path.join(train_folder, class_folder)
        for file_name in os.listdir(class_path):
            if file_name.endswith('.txt'):
                file_path = os.path.join(class_path, file_name)
                with open(file_path, 'r') as file:
                    train_data.append(file.read())
                    train_labels.append(class_folder)

# Load test data
test_data = []
test_labels = []

test_folder = os.path.join(data_folder, 'test')
for class_folder in os.listdir(test_folder):
    if os.path.isdir(os.path.join(test_folder, class_folder)):
        class_path = os.path.join(test_folder, class_folder)
        for file_name in os.listdir(class_path):
            if file_name.endswith('.txt'):
                file_path = os.path.join(class_path, file_name)
                with open(file_path, 'r') as file:
                    test_data.append(file.read())
                    test_labels.append(class_folder)

# Define BERT model
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'
bert_model_path = '/kaggle/input/bert-model-1'  # Replace with the actual file path
bert_preprocess_model_path = '/kaggle/input/preocessor'  # Replace with the actual file path

# Load BERT model and preprocessing model
bert_preprocess_model = hub.KerasLayer(bert_preprocess_model_path)
bert_model = hub.KerasLayer(bert_model_path)

# Preprocess the data using BERT
train_preprocessed = bert_preprocess_model(train_data)
train_embeddings = bert_model(train_preprocessed)['pooled_output']

# Convert BERT embeddings to document-term matrix using CountVectorizer
vectorizer = CountVectorizer()
train_doc_term_matrix = vectorizer.fit_transform(train_data)

# Apply HDP for topic modeling
hdp = LatentDirichletAllocation(n_components=10, random_state=42)  # Adjust the number of components as needed
train_hdp_features = hdp.fit_transform(train_doc_term_matrix)

# Convert labels to numeric format
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Train a classifier on BERT + HDP features
classifier = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(train_hdp_features.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),  
    tf.keras.layers.Dropout(0.5),                  
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
classifier.fit(train_hdp_features, train_labels_encoded, epochs=20)

# Preprocess and classify test data
test_preprocessed = bert_preprocess_model(test_data)
test_embeddings = bert_model(test_preprocessed)['pooled_output']
test_doc_term_matrix = vectorizer.transform(test_data)
test_hdp_features = hdp.transform(test_doc_term_matrix)
predictions = classifier.predict(test_hdp_features)

# Convert predictions back to labels
predicted_labels_encoded = tf.argmax(predictions, axis=1)
predicted_labels = label_encoder.inverse_transform(predicted_labels_encoded.numpy())

# Evaluate the classifier
accuracy = tf.metrics.Accuracy()
accuracy.update_state(test_labels_encoded, predicted_labels_encoded)
loss = classifier.evaluate(test_hdp_features, test_labels_encoded, verbose=0)[0]
f1 = f1_score(test_labels, predicted_labels, average='weighted')
precision = precision_score(test_labels, predicted_labels, average='weighted')
confusion = confusion_matrix(test_labels, predicted_labels)

print('Accuracy:', accuracy.result().numpy())
print('Loss:', loss)
print('F1 Score:', f1)
print('Precision:', precision)
print(confusion)


Epoch 1/20
233/233 [==============================] - 2s 6ms/step - loss: 1.2293 - accuracy: 0.5363
Epoch 2/20
233/233 [==============================] - 2s 10ms/step - loss: 1.0985 - accuracy: 0.5969
Epoch 3/20
233/233 [==============================] - 2s 9ms/step - loss: 1.0712 - accuracy: 0.5996
Epoch 4/20
233/233 [==============================] - 2s 9ms/step - loss: 1.0516 - accuracy: 0.6012
Epoch 5/20
233/233 [==============================] - 2s 8ms/step - loss: 1.0322 - accuracy: 0.6026
Epoch 6/20
233/233 [==============================] - 2s 9ms/step - loss: 1.0350 - accuracy: 0.6006
Epoch 7/20
233/233 [==============================] - 2s 8ms/step - loss: 1.0196 - accuracy: 0.6049
Epoch 8/20
233/233 [==============================] - 2s 8ms/step - loss: 1.0093 - accuracy: 0.6042
Epoch 9/20
233/233 [==============================] - 2s 7ms/step - loss: 1.0110 - accuracy: 0.6064
Epoch 10/20
233/233 [==============================] - 1s 3ms/step - loss: 1.0090 - accuracy: 0.605

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
